# Configuring your computer to use Python for scientific computing

*This document was generated from a Jupyter notebook, which you can download [here](configure_your_computer.ipynb).*

<hr>

## Setting up a Pixi project

We will use [Pixi](https://pixi.sh/), a relatively new package manager that I have found very effective. Importantly, it does so in a project-based way. That is, for each project, you use Pixi to create and manage the packages needed for that project. Our "project" here is our data analysis/statistical inference course.

**Step 1:** Install Pixi. To install Pixi, you need access to the command line. For macOS users, hit `Command-space`, type in "terminal" and open the `Terminal` app. In Windows, open PowerShell by opening the Start Menu, typing "PowerShell" in the search bar, and selecting "Windows PowerShell." I assume you know how to get access to the command line if you are using Linux.

On the command line, do the following.

**macOS or Linux**

    curl -fsSL https://pixi.sh/install.sh | sh

**Windows**

    powershell -ExecutionPolicy ByPass -c "irm -useb https://pixi.sh/install.ps1 | iex"


**Step 2:** Create a directory for your work in the course. You might want to name the directory `bebi25/`, which is what I have named it. You can do this either with the command line of your graphical file management program (e.g., Finder for macOS).

**Step 3:** Navigate to the directory you created on the command line. For example, if the directory is `bebi25/` in your home directory and you are in your home directory, you can do

    cd bebi25

on the command line.

**Step 4:** Download the requisite Pixi file: [pixi.toml](https://be25.github.io/handouts/pixi.toml). This file needs to be stored in the directory you created in step 3. You may download it by right-clicking this link ([pixi.toml](https://be25.github.io/handouts/pixi.toml)) or by doing the following on the command line.

**macOS or Linux**

    curl -fsSL https://be25.github.io/handouts/pixi.toml

**Windows**

    irm -useb https://be25.github.io/handouts/pixi.toml

**Step 5:** Install the environment! Do the following on the command line.

    pixi install

**Step 6:** To be able to use all of the packages, you need to invoke a Pixi shell. To do so, execute the following on the command line.

    pixi shell

You are now good to go! After you are done working, to exit the Pixi shell, hit `Control-D`.

**For doing work for this class, you will need to `cd` into the directory you created in step 2 and execute `pixi shell` every time you open a new terminal (or PowerShell) window.**


## Launching JupyterLab

You can launch JupyterLab is from the command line. To launch JupyterLab, type the following on the command line (again, *after* you have executed `pixi shell`).

    jupyter lab
    
You can specify the browser you want using, for example,

    jupyter lab --browser=firefox

Within the JupyterLab window, you will have the option to launch a notebook, a console, a terminal, or a text editor.

## Checking your distribution

With JupyterLab running, use the JupyterLab launcher (you can get a new launcher by clicking on the `+` icon on the left pane of your JupyterLab window) to launch a notebook. In the first cell (the box next to the `[ ]:` prompt), paste the code below. To run the code, press `Shift+Enter` while the cursor is active inside the cell. It may take a few moments for the cell to run. When it's done, you should see a plot that looks like the one below. If you do, you have a functioning Python environment for scientific computing!

In [1]:
import numpy as np
import pandas as pd

import eqtk

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

# Chemical reactions for aspartic acid
rxns = """
     <=> OH⁻ + H⁺  ; 1e-14
H₃D⁺ <=> H₂D + H⁺  ; {Ka1}
H₂D  <=> HD⁻ + H⁺  ; {Ka2}
HD⁻  <=> D²⁻ + H⁺  ; {Ka3}
""".format(Ka1=10**(-1.99), Ka2=10**(-3.9), Ka3=10**(-10.002))

# Stoichiometric matrix
N = eqtk.parse_rxns(rxns)

# Initial concentrations
c0 = {
    "H⁺":   0,
    "OH⁻":  0,
    "H₃D⁺": 0,
    "H₂D":  0.001,
    "HD⁻":  0,
    "D²⁻":  0,
}

# Set up pH variable
fixed_c = pd.DataFrame(data=-np.ones((400, 6)), columns=c0.keys())
pH = np.linspace(0, 14, 400)
fixed_c["H⁺"] = 10**(-pH)

# Solve for equilibrium concentrations
c = eqtk.fixed_value_solve(c0=c0, fixed_c=fixed_c, N=N, units='M')

# Parse concentrations
c['pH'] = -np.log10(c['[H⁺] (M)'])
c['H₃D⁺'] = c['[H₃D⁺] (M)'] / c0["H₂D"]
c['H₂D'] = c['[H₂D] (M)'] / c0["H₂D"]
c['HD⁻'] = c['[HD⁻] (M)'] / c0["H₂D"]
c['D²⁻'] = c['[D²⁻] (M)'] / c0["H₂D"]

# Make the plot
p = bokeh.plotting.figure(
    height=250,
    width=420,
    y_axis_label="relative abundance",
    x_axis_label="pH",
    x_range=[0, 14],
)

p.line(c['pH'], c['H₃D⁺'], color='#4c78a8', line_width=2, legend_label="H₃D⁺")
p.line(c['pH'], c['H₂D'], color='#f58518', line_width=2, legend_label="H₂D")
p.line(c['pH'], c['HD⁻'], color='#e45756', line_width=2, legend_label="HD⁻")
p.line(c['pH'], c['D²⁻'], color='#72b7b2', line_width=2, legend_label="D²⁻")

p.legend.location = 'center_right'

bokeh.io.show(p)

Loading BokehJS ...

## Computing environment

In [2]:
%load_ext watermark
%watermark -v -p numpy,pandas,eqtk,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.13.11
IPython version      : 9.8.0

numpy     : 2.2.6
pandas    : 2.3.3
eqtk      : 0.1.4
bokeh     : 3.8.1
jupyterlab: 4.5.1

